### Scalability
This notebook aims to find out the best way to make the implemented SolarBAG system scalable.

In [1]:
# Imports
import json
import sys

from cjio import cityjson
from os import listdir
from os.path import isfile, join
from rtree import index

import utils

**Step 1** <br>
Choose 9 equal-sized tiles positioned in a 3x3 matrix relative from each other. <br>
Download them as CityJSON and store them in a folder.

**Step 2** <br>
Load the metadata of each tile in the folder and extract the 'geographical extent' attribute.

In [2]:
# Put the CityJSON files from a path referring to a folder in a list.
path_folder = "C:\\Users\\hurkm\\Documents\\tiles_temp\\"
cj_files = [file for file in listdir(path_folder) if isfile(join(path_folder, file))]

In [3]:
# Note: the slow approach
# Loop over the files in the list to load them and extract the bbox.
for file in cj_files:
    path_file = path_folder + file

    # Load the city model (cm) from the path to the file
    cm = cityjson.load(path_file)
    bbox = cm.get_bbox()
    print(file, bbox)

3dbag_v210908_fd2cee53_5801.json [78810.8410032959, 454373.10899487307, -0.859077513217926, 79462.87300097656, 455058.7599958496, 31.883384704589844]
3dbag_v210908_fd2cee53_5802.json [78810.99400027466, 453801.2899960937, -2.057142734527588, 79462.598015625, 454439.4409970703, 33.40597915649414]
3dbag_v210908_fd2cee53_5805.json [78812.13499731445, 453182.9969887695, -1.1712751388549805, 79448.33900732422, 453818.2179970703, 28.69548797607422]
3dbag_v210908_fd2cee53_5872.json [79427.34199993896, 453167.90300549316, -0.36000001430511475, 80071.09801953124, 453821.61398876953, 58.812522888183594]
3dbag_v210908_fd2cee53_5873.json [80051.07700390625, 453174.41800756834, -0.23687732219696045, 80701.49100537109, 453822.2850119629, 43.109031677246094]
3dbag_v210908_fd2cee53_5875.json [79425.54099902343, 453798.1269765625, -1.1979999542236328, 80096.66099414062, 454445.5050016174, 28.436384201049805]
3dbag_v210908_fd2cee53_5876.json [79424.7909998474, 454419.948, -0.5986470580101013, 80082.3219

In [4]:
# Loop over the files in the list with tiles and search for the metadata object to extract the geographical extent (bbox) of the tile.
tile_bboxes = []
for file in cj_files:
    path_file = path_folder + file
    f = open(path_file, "r")
    s = f.read()
    #-- find "metadata"
    posm = s.find("metadata")
    pos_start = s.find("{", posm)
    pos_end = 0
    cur = pos_start
    count = 1
    while True:
        a = s.find("{", cur+1) 
        b = s.find("}", cur+1) 
        if a < b: 
            count += 1
            cur = a
        else: 
            count -= 1
            cur = b
        if count == 0:
            pos_end = b
            break
    m = s[pos_start:pos_end+1]
    jm = json.loads(m)

    c1 = jm['geographicalExtent'][0:2]
    c2 = jm['geographicalExtent'][3:5]
    bbox_2D = c1 + c2

    # print(bbox_2D)
    tile_bboxes.append((path_file, bbox_2D))

print(tile_bboxes)


[('C:\\Users\\hurkm\\Documents\\tiles_temp\\3dbag_v210908_fd2cee53_5801.json', [78810.8410032959, 454373.10899487307, 79462.87300097656, 455058.7599958496]), ('C:\\Users\\hurkm\\Documents\\tiles_temp\\3dbag_v210908_fd2cee53_5802.json', [78810.99400027466, 453801.2899960937, 79462.598015625, 454439.4409970703]), ('C:\\Users\\hurkm\\Documents\\tiles_temp\\3dbag_v210908_fd2cee53_5805.json', [78812.13499731445, 453182.9969887695, 79448.33900732422, 453818.2179970703]), ('C:\\Users\\hurkm\\Documents\\tiles_temp\\3dbag_v210908_fd2cee53_5872.json', [79427.34199993896, 453167.90300549316, 80071.09801953124, 453821.61398876953]), ('C:\\Users\\hurkm\\Documents\\tiles_temp\\3dbag_v210908_fd2cee53_5873.json', [80051.07700390625, 453174.41800756834, 80701.49100537109, 453822.2850119629]), ('C:\\Users\\hurkm\\Documents\\tiles_temp\\3dbag_v210908_fd2cee53_5875.json', [79425.54099902343, 453798.1269765625, 80096.66099414062, 454445.5050016174]), ('C:\\Users\\hurkm\\Documents\\tiles_temp\\3dbag_v210908

As seen in the two code blocks above, it is much faster to search for the metadata object to extract the bbox instead of loading the whole CityJSON file into memory and then get the bbox. <br>
Code of the second block is taken from https://github.com/cityjson/specs/issues/67#issuecomment-654202473

**Step 3** <br>
Create an R-Tree that stores the 2D bounding boxes of each tile.

In [5]:
 # Set properties for the rtree index.
p = index.Property()
p.dimension = 2

# Create empty rtree for the bounding boxes of a tile
rtree_idx_tile = index.Index(properties=p)

# Loop to dump all tile's bboxes in the rtree.
for i, tile in enumerate(tile_bboxes, 0):
    path = tile[0]
    bbox = tile[1]
    # Insert the bbox into the rtree with the tile's file path as object (obj).
    rtree_idx_tile.insert(i, bbox, obj=path)

print("Size of the Rtree: {}".format(rtree_idx_tile.get_size()))
print(rtree_idx_tile)

Size of the Rtree: 10
rtree.index.Index(bounds=[78810.8410032959, 453167.90300549316, 81332.01200006103, 455074.5599987793], size=10)


**Step 4** <br>
Loop over each tile in the folder and find their 8 neighbouring tiles by querying the 'rtree_idx_tile'. <br>
Then, read these neighbouring tiles including the current tile completely and store the geometries of the buildings within a buffer of, say 100m, in the rtree that is already created in SolarBAG.

In [6]:
# This function gradually builds the dictionary with all potential neighbours for a tile.
# This includes buildings from neighbouring tiles.
def build_nb_dict(nb_buildings, cm_nb, buffer_box):
    # Create an r-tree of cm_nb'
    buildings = cm_nb.get_cityobjects(type='BuildingPart')

    lod = "2.2"     # In solarBAG this is already initialised.
    rtree_idx_nb = utils.create_rtree(buildings, lod)
    print(rtree_idx_nb)

    # Adjust 2D buffer box to 3D to be compatible with 3D rtree.
    buffer_box_3d = [buffer_box[0], buffer_box[1], -100, buffer_box[2], buffer_box[3], 100]

    # Find the buildings in each neighbouring tile that is situated within the buffer box limits.
    hits = list(rtree_idx_nb.intersection(buffer_box_3d, objects="True"))
    print(len(hits))

    # Put all the buildings in hits in the nb_buildings dict
    for item in hits:
        bdg_id = item.object
        bdg_obj = buildings[bdg_id]

        # Add the building from a neighbouring tile to the nb_buildings dict
        # Use its id as key and the actual building object as value.
        nb_buildings[bdg_id] = bdg_obj
    

    return nb_buildings

In [7]:
for tile in tile_bboxes:
    path = tile[0]
    bbox = tile[1]
    print("Finding neighbours of tile: {}".format(path.split("\\")[5]))

    hits = list(rtree_idx_tile.intersection([bbox[0]-100, bbox[1]-100, bbox[2]+100, bbox[3]+100], objects="True"))

    # Next step: for each neighbouring tile, store the buildings with a buffer of 100m. 
    # For this, it is needed to store all the buildings of that tile in an Rtree.
    # So, maybe skip incorporating neighbouring tiles in shadow casting for now and focus on scalability only?
    for item in hits:
        path_nb = item.object
        if path_nb != path:
            # print(path_nb)

            cm = cityjson.load(path_nb)
    

Finding neighbours of tile: 3dbag_v210908_fd2cee53_5801.json
Finding neighbours of tile: 3dbag_v210908_fd2cee53_5802.json
Finding neighbours of tile: 3dbag_v210908_fd2cee53_5805.json
Finding neighbours of tile: 3dbag_v210908_fd2cee53_5872.json
Finding neighbours of tile: 3dbag_v210908_fd2cee53_5873.json
Finding neighbours of tile: 3dbag_v210908_fd2cee53_5875.json
Finding neighbours of tile: 3dbag_v210908_fd2cee53_5876.json
Finding neighbours of tile: 3dbag_v210908_fd2cee53_5877.json
Finding neighbours of tile: 3dbag_v210908_fd2cee53_5878.json
Finding neighbours of tile: 3dbag_v210908_fd2cee53_5880.json


**Step 4 (alternative 1)** <br>
This is is not implemented as it probably will exceed memory capacities. <br>
Load the city model of all tiles in memory as a dictionary with their path as the key and the city model as the value. <br>
Loop over each tile in the folder and find their 8 neighbouring tiles by querying the 'rtree_idx_tile'. This returns the path to the tile. <br>
Then, look up the city model corresponding to the path in the dictionary and store the geometries of the buildings within a buffer of, say 100m, in the rtree that is already created in SolarBAG.

**Step 4 (alternative 2)** <br>
Loop over each tile in the folder and find their 8 or less neighbouring tiles by querying the 'rtree_idx_tile'. <br>
Store the path together with their city model in a dictionary if it does not yet exist in the dictionary. This prevents loading the tile a second time.<br>
Then, store the geometries of the buildings in the neighbouring tile within a buffer of, say 100m, in the rtree that is already created in SolarBAG. <br>
Note that this method is faster **but** the memory usage will gradually grow when doing this. Maybe find a way to delete city models from the dictionary when not needed anymore?

In [8]:
nb_dict = {}

for tile in tile_bboxes:
    path = tile[0]
    bbox = tile[1]
    print("Finding neighbours of tile: {}".format(path.split("\\")[5]))

    # Load the path of the current tile if it is not yet stored in nb_dict
    if path in nb_dict:
        cm = nb_dict[path]
    else:
        cm = cityjson.load(path)
        nb_dict[path] = cm

    # Ideally triangulate the city model after loading. NOTE: This takes looooong.
    try:
        cm.triangulate()
    except:
        print("Tile {} could not be triangulated. \nThis tile will be skipped.".format(path))
        continue

    # Get the buildings as BuildingPart (these contain LoD above 0) from the city model as dict (there are only buildings).
    buildings = cm.get_cityobjects(type='BuildingPart')
    # print(len(buildings))
    nb_buildings = buildings    # copy the buildings dict into the nb_buildings dict where buildings from other tiles will also be inserted.

    # Find the tiles that are direct neighbours of the current tile.
    buffer_box = [bbox[0]-100, bbox[1]-100, bbox[2]+100, bbox[3]+100]
    hits = list(rtree_idx_tile.intersection(buffer_box, objects="True"))

    # Next step: for each neighbouring tile, store the buildings with a buffer of 100m. 
    # For this, it is needed to store all the buildings of that tile in an Rtree.
    # So, maybe skip incorporating neighbouring tiles in shadow casting for now and focus on scalability only?
    for item in hits:
        path_nb = item.object
        if path_nb != path:
            if path_nb not in nb_dict:
                print(path_nb)
                
                cm_nb = cityjson.load(path_nb)
                nb_dict[path_nb] = cm_nb
            else:
                print(path_nb)
                cm_nb = nb_dict[path_nb]
            
            nb_buildings = build_nb_dict(nb_buildings, cm_nb, buffer_box)
        # Create an Rtree for the neighbouring city model.
        # Query the buffer bbox and get the buildings situated in this buffer.
        # Return their building geometries and add them to the already created buildings dictionary for which an rtree can be created.
        # Then process as already implemented.
        # Hopefully this will reduce memory usage.
    print(len(nb_buildings.keys()))     # This shows that the nb_buildings dict contains the buildings from the current tile and buildings from within the buffer box out of neighbouring tiles.
    
    # Create the rtree including all potential neighbouring blocking buildings
    rtree_idx_bdg = utils.create_rtree(nb_buildings, "2.2")

    # PROCESS FURTHER AS ALREADY IMPLEMENTED IN SOLARBAG.
    
    # print(nb_dict.keys())

Finding neighbours of tile: 3dbag_v210908_fd2cee53_5801.json


c:\Users\hurkm\AppData\Local\Programs\Python\Python39\lib\site-packages\cjio\geom_help.py:59: RuntimeWarning: overflow encountered in long_scalars
  n[1] += ( (poly[i][2] - poly[ne][2]) * (poly[i][0] + poly[ne][0]) )
c:\Users\hurkm\AppData\Local\Programs\Python\Python39\lib\site-packages\cjio\geom_help.py:60: RuntimeWarning: overflow encountered in long_scalars
  n[2] += ( (poly[i][0] - poly[ne][0]) * (poly[i][1] + poly[ne][1]) )


C:\Users\hurkm\Documents\tiles_temp\3dbag_v210908_fd2cee53_5802.json
Size of the Rtree: 1620
rtree.index.Index(bounds=[78810.994, 453801.29, -2.057, 79462.595, 454439.441, 33.405], size=1620)
526
C:\Users\hurkm\Documents\tiles_temp\3dbag_v210908_fd2cee53_5875.json
Size of the Rtree: 1236
rtree.index.Index(bounds=[79425.541, 453798.127, -1.198, 80096.661, 454445.505, 28.436], size=1236)
173
C:\Users\hurkm\Documents\tiles_temp\3dbag_v210908_fd2cee53_5876.json
Size of the Rtree: 955
rtree.index.Index(bounds=[79424.81899999999, 454419.975, -0.598, 80082.322, 455059.071, 81.72500000000001], size=955)
326
2760
Size of the Rtree: 2760
Finding neighbours of tile: 3dbag_v210908_fd2cee53_5802.json
C:\Users\hurkm\Documents\tiles_temp\3dbag_v210908_fd2cee53_5801.json
Size of the Rtree: 1735
rtree.index.Index(bounds=[78810.841, 454373.23600000003, -0.859, 79462.847, 455058.672, 31.883], size=1735)
267
C:\Users\hurkm\Documents\tiles_temp\3dbag_v210908_fd2cee53_5805.json
Size of the Rtree: 1688
rtree

c:\Users\hurkm\AppData\Local\Programs\Python\Python39\lib\site-packages\cjio\geom_help.py:58: RuntimeWarning: overflow encountered in long_scalars
  n[0] += ( (poly[i][1] - poly[ne][1]) * (poly[i][2] + poly[ne][2]) )


C:\Users\hurkm\Documents\tiles_temp\3dbag_v210908_fd2cee53_5801.json
Size of the Rtree: 1735
rtree.index.Index(bounds=[78810.841, 454373.23600000003, -0.859, 79462.847, 455058.672, 31.883], size=1735)
58
C:\Users\hurkm\Documents\tiles_temp\3dbag_v210908_fd2cee53_5802.json
Size of the Rtree: 1620
rtree.index.Index(bounds=[78810.994, 453801.29, -2.057, 79462.595, 454439.441, 33.405], size=1620)
284
C:\Users\hurkm\Documents\tiles_temp\3dbag_v210908_fd2cee53_5805.json
Size of the Rtree: 1688
rtree.index.Index(bounds=[78812.135, 453182.997, -1.171, 79448.339, 453818.218, 28.695], size=1688)
62
C:\Users\hurkm\Documents\tiles_temp\3dbag_v210908_fd2cee53_5872.json
Size of the Rtree: 1724
rtree.index.Index(bounds=[79431.643, 453167.903, -0.36, 80071.098, 453821.614, 58.812000000000005], size=1724)
218
C:\Users\hurkm\Documents\tiles_temp\3dbag_v210908_fd2cee53_5873.json
Size of the Rtree: 1400
rtree.index.Index(bounds=[80051.077, 453174.418, -0.23600000000000002, 80701.476, 453822.285, 43.109], 

**Memory usage findings** <br>
After running the code block (of step 4, alternative 2) above, it can be observed that the memory usage gradually increases further in the process. But it does not exceed 13GB.